In [1]:
import sys  
from PyQt5 import QtWidgets,QtCore, QtGui
from PyQt5.QtWidgets import QApplication, QMainWindow,QWidget, QGridLayout, QHBoxLayout, QVBoxLayout
from PyQt5 import QtWebEngineWidgets
from PyQt5.QtGui import QPixmap
from mainWindow import Ui_MainWindow #這個是main   主視窗
import subWindow 
import webWindow 
import pandas as pd
import DataFunction as df
from datetime import datetime

class MyMain(QMainWindow,Ui_MainWindow):    #主視窗
    def __init__(self):
        super().__init__()
        self.setupUi(self) 
        self.setWindowTitle('壓力測試主畫面')
        self.setFixedSize(551, 462)
        
        # 日期 
        if datetime.today().weekday() != 0:    
            self.MWDE_1.setDateTime(QtCore.QDateTime.currentDateTime().addDays(-1))
        else:
            self.MWDE_1.setDateTime(QtCore.QDateTime.currentDateTime().addDays(-3))
        # 初始化 
        self.onDateChanged()
        # 操作 
        self.MWDE_1.dateChanged.connect(self.onDateChanged)
#-------------------------------------------------------------------------------------------------------------#
    def onDateChanged(self):    # 轉換時間
        datetime = self.MWDE_1.dateTime().toString("yyyy/MM/dd")
        usd = df.Equ_Mar_Ratio_All('USD',datetime)
        jpy = df.Equ_Mar_Ratio_All('JPY',datetime)
        hkd = df.Equ_Mar_Ratio_All('HKD',datetime)
        eud = df.Equ_Mar_Ratio_All('EUD',datetime)
        self.MWLB_U2.setText(f'{usd[0]:,d} / {usd[1]:,d}  ({usd[2]})')
        self.MWLB_J2.setText(f'{jpy[0]:,d} / {jpy[1]:,d}  ({jpy[2]})')
        self.MWLB_H2.setText(f'{hkd[0]:,d} / {hkd[1]:,d}  ({hkd[2]})')
        self.MWLB_E2.setText(f'{eud[0]:,d} / {eud[1]:,d}  ({eud[2]})')
        df.pie_equity(datetime)      # 產圖(df.pie_equity)
        self.MWLB_1.setPixmap(QPixmap("pie_eqt.png"))
        self.MWLB_1.setScaledContents (True)
#-------------------------------------------------------------------------------------------------------------#        
    def closeEvent(self, event):    # 連動關閉視窗
        QApplication.closeAllWindows()

###############################################################################################################
class WidgetP(QWidget,webWindow.Ui_Form):    #圖形分析
    def __init__(self):
        super().__init__()
        self._widget = QWidget()
        self.setupUi(self)  
        self.setFixedSize(1000,800)
        self.setWindowTitle('圖形分析')

        self.pie = QtWidgets.QPushButton('圓餅圖', self)
        self.bar = QtWidgets.QPushButton('長條圖', self)
        self.line = QtWidgets.QPushButton('折線圖', self)
        self.browser = QtWebEngineWidgets.QWebEngineView(self)
 
        grid_layout = QtWidgets.QGridLayout()    #網格排列
        self.setLayout(grid_layout)
        grid_layout.addWidget(self.pie, 0, 0)
        grid_layout.addWidget(self.bar, 0, 1)
        grid_layout.addWidget(self.line, 0, 2)
        grid_layout.addWidget(self.browser, 1,0,1,3)
        
        # 操作 ---------------------------------------------------------------------------------
        self.pie.clicked.connect(self.sunburst_margin)
        self.bar.clicked.connect(self.bar_oi)
        self.line.clicked.connect(self.line_lot)
        
        #圖型
#-------------------------------------------------------------------------------------------------------------#
    def sunburst_margin(self):
        self.browser.setHtml(df.sunburst_margin_use())
#-------------------------------------------------------------------------------------------------------------#        
    def bar_oi(self):
        self.browser.setHtml(df.bar_oi_use())
#-------------------------------------------------------------------------------------------------------------#            
    def line_lot(self):
        self.browser.setHtml(df.line_lot_use())
###############################################################################################################  
class WidgetR(QWidget,subWindow.Ui_Form):    #壓力測試
    def __init__(self):
        super().__init__()
        self.setupUi(self)  # 初始化執行B視窗類下的 setupUi 函式
        self.setFixedSize(587, 746)
        self.setWindowTitle('壓力測試')
        
        #  日期設定
        shs = (pd.ExcelFile('RiskControl.xlsx').sheet_names)[-1]
        date_1 = shs[:4] + '/' + shs[5:7] + '/' + shs[7:9] 
        self.SWLB_D1.setText(date_1)
        
        #  上手設定
        self.SWCB_1.currentIndexChanged.connect(self.onBrokerChange)
        self.SWCB_1.currentIndexChanged.connect(self.clear_all)
        
        #  全部清除
        self.SWPB_1.clicked.connect(self.clear_all)


        # #觸發條件 ( 上手變動 )
#-------------------------------------------------------------------------------------------------------------#
    def onBrokerChange(self):
        print('onBrokerChange')
        
        # 斷開連線，避免格式化時互相影響。
        self.SWCB_2.disconnect()
        self.SWCB_3.disconnect()
        self.SWCB_4.disconnect()
        self.SWCB_5.disconnect()
        self.SWCB_6.disconnect()
        
        datetime = self.SWLB_D1.text()
        broker = self.SWCB_1.currentText()
        # 調整前 (權益數/保證金/風險) 輸出
        usd = df.Equ_Mar_Ratio_Each('USD', datetime, broker)
        jpy = df.Equ_Mar_Ratio_Each('JPY', datetime, broker)
        hkd = df.Equ_Mar_Ratio_Each('HKD', datetime, broker)
        eud = df.Equ_Mar_Ratio_Each('EUD', datetime, broker)
        self.SWLB_U2.setText(f'{usd[0]:,d} / {usd[1]:,d}  ({usd[2]})')
        self.SWLB_J2.setText(f'{jpy[0]:,d} / {jpy[1]:,d}  ({jpy[2]})')
        self.SWLB_H2.setText(f'{hkd[0]:,d} / {hkd[1]:,d}  ({hkd[2]})')
        self.SWLB_E2.setText(f'{eud[0]:,d} / {eud[1]:,d}  ({eud[2]})')
        
        # 取得前五商品
        future = df.Up_3(broker)
        
        # 加入第一樣商品為空白    
        products = (' ',) + future[0]
        
        # 清除 ComboBox 前一個選取上手資料
        self.SWCB_2.clear() 
        self.SWCB_3.clear()
        self.SWCB_4.clear()
        self.SWCB_5.clear()
        self.SWCB_6.clear()
        
        # ComboBox 重新加入上手商品
        self.SWCB_2.addItems(products)
        self.SWCB_3.addItems(products)
        self.SWCB_4.addItems(products)
        self.SWCB_5.addItems(products)
        self.SWCB_6.addItems(products)
        
        # 結算價變空白
        self.SWLB_6.setText(' ')
        self.SWLB_7.setText(' ')
        self.SWLB_8.setText(' ')
        self.SWLB_9.setText(' ')
        self.SWLB_10.setText(' ')
        
        # 連結 ComboBox ( 商品變動 > 輸出結算價 ) 
        print('SWCB_2 連線1')
        self.SWCB_2.currentIndexChanged.connect(lambda: self.onProductChange_1(self.SWCB_2, self.SWLB_6))
        print('SWCB_3 連線1')
        self.SWCB_3.currentIndexChanged.connect(lambda: self.onProductChange_1(self.SWCB_3, self.SWLB_7))
        print('SWCB_4 連線1')
        self.SWCB_4.currentIndexChanged.connect(lambda: self.onProductChange_1(self.SWCB_4, self.SWLB_8))
        print('SWCB_5 連線1')
        self.SWCB_5.currentIndexChanged.connect(lambda: self.onProductChange_1(self.SWCB_5, self.SWLB_9))
        print('SWCB_6 連線1')
        self.SWCB_6.currentIndexChanged.connect(lambda: self.onProductChange_1(self.SWCB_6, self.SWLB_10))
        
        # 連結 ComboBox ( 商品變動 > Spinbox,Slider 歸零 (輸出調整後權益數) > *啟用 Spinbox, Slider 連結 )
        print('SWCB_2 連線2')
        self.SWCB_2.currentIndexChanged.connect(lambda: self.onProductChange_2(self.SWSBR_1, self.SWLB_6, self.SWSR_1, self.SWCB_2))
        print('SWCB_3 連線2')
        self.SWCB_3.currentIndexChanged.connect(lambda: self.onProductChange_2(self.SWSBR_2, self.SWLB_7, self.SWSR_2, self.SWCB_3))
        print('SWCB_4 連線2')
        self.SWCB_4.currentIndexChanged.connect(lambda: self.onProductChange_2(self.SWSBR_3, self.SWLB_8, self.SWSR_3, self.SWCB_4))
        print('SWCB_5 連線2')
        self.SWCB_5.currentIndexChanged.connect(lambda: self.onProductChange_2(self.SWSBR_4, self.SWLB_9, self.SWSR_4, self.SWCB_5))
        print('SWCB_6 連線2')
        self.SWCB_6.currentIndexChanged.connect(lambda: self.onProductChange_2(self.SWSBR_5, self.SWLB_10, self.SWSR_5, self.SWCB_6))
        
    
        #觸發條件 ( 重新整理 or 上手變動 )
#-------------------------------------------------------------------------------------------------------------#       
    def clear_all(self):
        print('clear_all')
        
        # 商品變空白  > 1.進入 onProductChange_1() > 結算價變空白 
        #            > 2.進入 onProductChange_2() > 啟用 Spinbox, Slider 連結 ， 避免重新連結會先try，如已經連結會線斷線再重連。
        self.SWCB_2.setCurrentText(' ')
        self.SWCB_3.setCurrentText(' ')
        self.SWCB_4.setCurrentText(' ')
        self.SWCB_5.setCurrentText(' ')
        self.SWCB_6.setCurrentText(' ')
        
        print('調整設定前')
        # 將調整後權益數設為預設值
        self.SWLB_U4.setText(self.SWLB_U2.text())
        self.SWLB_J4.setText(self.SWLB_J2.text())
        self.SWLB_H4.setText(self.SWLB_H2.text())
        self.SWLB_E4.setText(self.SWLB_E2.text())
        print('調整設定後')
        
        # 將 Spinbox, Slider 歸零
        self.SWSBR_1.setProperty("value", 0)
        self.SWSBR_2.setProperty("value", 0)
        self.SWSBR_3.setProperty("value", 0)
        self.SWSBR_4.setProperty("value", 0)
        self.SWSBR_5.setProperty("value", 0)
        self.SWSR_1.setProperty("value", 0)
        self.SWSR_2.setProperty("value", 0)
        self.SWSR_3.setProperty("value", 0)
        self.SWSR_4.setProperty("value", 0)
        self.SWSR_5.setProperty("value", 0)
        
        # 5格損益重新設定
        self.diff_1, self.diff_2, self.diff_3, self.diff_4, self.diff_5 = (0, ''), (0, ''), (0, ''), (0, ''), (0, '')
 

        # 觸發條件 ( 重新整理 or 上手變動 or 換選其他商品 )
#-------------------------------------------------------------------------------------------------------------#       
    def onProductChange_1(self, ComboBox, Label):
        print('onProductChange_1')
        
        # 如遇選擇空白時，不用計算直接輸出空白。
        if ComboBox.currentText() == " ":                
            Label.setText(' ')        
            return
        #取上手
        broker = self.SWCB_1.currentText()               
        
        # 取商品
        product = ComboBox.currentText()
        
        # 取佔比大於3%商品 > DataFuction.py 的函示  
        future = df.Up_3(broker)
        
        # 取商品相應結算價位置  future = ((商品), (結算價), (淨口數))
        n = future[0].index(product)
        
        # 處理當結算價取到分數形式，並輸出價格 (應放在前面資料處理的程式碼)
        if '/' in future[1][n]:                          
            fraction = (future[1][n]).split(" ")                  
            fraction_D = fraction[1].split('/')                  
            price = float(fraction[0]) + float(fraction_D[0]) / float(fraction_D[1])
            Label.setText(f'{price:.2f}')                        
        else:
            Label.setText(future[1][n])                  
        
        # 觸發條件 ( 重新整理 or 上手變動 or 換選其他商品 )
#-------------------------------------------------------------------------------------------------------------#       
    def onProductChange_2(self, SpinBox, Label, SliderBox, ComboBox):
        print('onProductChange_2')
        
        # 初始化價格文字設定 ( 可能受前一筆影響，為紅字 or 綠字)
        Label.setStyleSheet("QLabel{border:1px; border-style:solid; color:black}")
        
        # SpinBox 位置歸 0 ， SliderBox 位置歸 0 再底下，如一起放在這會造成重複輸出權益數
        SpinBox.setProperty("value", 0)              
        print('SpinBox 位置歸零')
        
        # 因為一開始 Spinbox,Slider 沒有連線，需要透過選完 ComboBox 再連線，但避免重複連線，故先 Try，如重覆則先斷線再連回。
        try:
            SpinBox.valueChanged.disconnect()
            print('SpinBox 原本有連線，先斷線')
            SpinBox.valueChanged.connect(lambda: self.onPriceChange_Spin(SpinBox, Label, SliderBox, ComboBox)) # Spinbox > 結算價、Slider
            print('SpinBox 再接回來')
        except:
            SpinBox.valueChanged.connect(lambda: self.onPriceChange_Spin(SpinBox, Label, SliderBox, ComboBox)) # Spinbox > 結算價、Slider
            print('SpinBox 原本沒有連線，先連線')
        
        try:
            SliderBox.valueChanged.disconnect()
            print('SliderBox 原本有連線，先斷線')
            SliderBox.setProperty("value", 0)           
            print('SliderBox 位置歸零')
            SliderBox.valueChanged.connect(lambda: self.onPriceChange_Spin(SpinBox, Label, SliderBox, ComboBox)) # Spinbox > 結算價、Slider
            print('SliderBox 再接回來') 
        except:
            SliderBox.setProperty("value", 0)            
            print('SliderBox 位置歸零')
            SliderBox.valueChanged.connect(lambda: self.onPriceChange_Slider(SpinBox, Label, SliderBox, ComboBox)) # Slider > 結算價、Spinbox 
            print('SliderBox 原本沒有連線，先連線')
            

#-------------------------------------------------------------------------------------------------------------#       
        # 觸發條件 Slider 移動
    def onPriceChange_Slider(self, SpinBox, Label, SliderBox, ComboBox):
        print('onPriceChange_Slider')
        
        # 如是更新此單一商品，不須計算，直接判斷是哪一欄，並輸出損益為 0 幣別為空白 EX (0, "")
        if ComboBox.currentText() == " ":                            
            if ComboBox ==  self.SWCB_2:
                print('diff_1 = 0')
                self.diff_1 = (0, '')
            elif ComboBox ==  self.SWCB_3:
                print('diff_2 = 0')
                self.diff_2 = (0, '')
            elif ComboBox ==  self.SWCB_4:
                print('diff_3 = 0')
                self.diff_3 = (0, '')
            elif ComboBox ==  self.SWCB_5:
                print('diff_4 = 0')
                self.diff_4 = (0, '')
            else: 
                print('diff_5 = 0')
                self.diff_5 = (0, '')
            self.output_profit_and_loss()
            return
        broker = self.SWCB_1.currentText()                           # 取上手
        product = ComboBox.currentText()                             # 取商品
        self.onProductChange_1(ComboBox, Label)                      # 初始化結算價
        SpinBox.disconnect()                                         # 斷開連結 避免slider
        print('SliderBox滑動，防止SpinBox改變，先斷線')
        SpinBox.setProperty("value", SliderBox.value())              # 設定Slider位置同步化
        SpinBox.valueChanged.connect(lambda: self.onPriceChange_Spin(SpinBox, Label, SliderBox, ComboBox)) #重新連結
        print('SpinBox連回來')
        p_num = df.percent_num(product)                              # 取小數後幾位(df.percent_num)
        price = float(Label.text()) * (1 + SliderBox.value() * 0.01) # 調整後價格
        diff = price - float(Label.text())                           # 取差價                        

        # 取佔比大於3%商品 > DataFuction.py 的函示  
        future = df.Up_3(broker)                         
        
        # 取口數 future = ((商品), (結算價), (淨口數))
        lot = future[2][future[0].index(product)]        
        
        # 算損益 & 幣別
        ProfitLoss, currency = df.profitloss_currency(product, diff, lot)
        
        # 分損益 & 幣別到相對應的欄位
        if ComboBox ==  self.SWCB_2:
            self.diff_1 = (ProfitLoss, currency)
        elif ComboBox ==  self.SWCB_3:
            self.diff_2 = (ProfitLoss, currency)
        elif ComboBox ==  self.SWCB_4:
            self.diff_3 = (ProfitLoss, currency)
        elif ComboBox ==  self.SWCB_5:
            self.diff_4 = (ProfitLoss, currency)
        else: 
            self.diff_5 = (ProfitLoss, currency)
        
        #輸出損益
        self.output_profit_and_loss()

        # 輸出商品結算價差異 EX:60.57 (+0.57)
        diff = ('(+' + f'{diff:.{p_num}f}' + ')') if  diff >= 0 else ('(' + f'{diff:.{p_num}f}' + ')')
        Label.setText(f'{price:.{p_num}f}{diff}')
        
        # 設定顏色格式 
        if SliderBox.value() > 0:                                    
            Label.setStyleSheet("QLabel{border:1px; border-style:solid; color:red}")
        elif SliderBox.value() < 0:
            Label.setStyleSheet("QLabel{border:1px; border-style:solid; color:green}")
        else:
            Label.setStyleSheet("QLabel{border:1px; border-style:solid; color:black}")
            Label.setText(f'{price:.{p_num}f}')                      # 差異為 0             

#-------------------------------------------------------------------------------------------------------------#       
        # 觸發條件 Spin 移動
    def onPriceChange_Spin(self, SpinBox, Label, SliderBox, ComboBox):
        print('onPriceChange_Spin')
        if ComboBox.currentText() == " ":                                      # 防止商品為空白
            if ComboBox ==  self.SWCB_2:
                print('diff_1 = 0')
                self.diff_1 = (0, '')
            elif ComboBox ==  self.SWCB_3:
                print('diff_2 = 0')
                self.diff_2 = (0, '')
            elif ComboBox ==  self.SWCB_4:
                print('diff_3 = 0')
                self.diff_3 = (0, '')
            elif ComboBox ==  self.SWCB_5:
                print('diff_4 = 0')
                self.diff_4 = (0, '')
            else :
                print('diff_5 = 0')
                self.diff_5 = (0, '') 
            self.output_profit_and_loss()
            return
        broker = self.SWCB_1.currentText()                           # 取上手
        product = ComboBox.currentText()                             # 取商品
        self.onProductChange_1(ComboBox, Label)                      # 初始化結算價
        
        SliderBox.disconnect()                                       # 斷開連結 避免slider
        print('SpinBox滑動，防止SliderBox改變，先斷線')
        SliderBox.setProperty("value", SpinBox.value())              # 設定Slider位置
        SliderBox.valueChanged.connect(lambda: self.onPriceChange_Slider(SpinBox, Label, SliderBox, ComboBox)) #重新連結
        print('SliderBox連回來')
        p_num = df.percent_num(product)                              # 取小數後幾位(df.percent_num)
        price = float(Label.text()) * (1 + SpinBox.value() * 0.01)   # 調整後價格
        diff = price - float(Label.text())                           # 取差價
        
        # 取佔比大於3%商品 > DataFuction.py 的函示   
        future = df.Up_3(broker)                         
        
        # 取口數 future = ((商品), (結算價), (淨口數))
        lot = future[2][future[0].index(product)]       
        
        # 算損益 & 幣別
        ProfitLoss, currency = df.profitloss_currency(product, diff, lot)
        
        # 分損益 & 幣別到相對應的欄位
        if ComboBox ==  self.SWCB_2:
            self.diff_1 = (ProfitLoss, currency)
        elif ComboBox ==  self.SWCB_3:
            self.diff_2 = (ProfitLoss, currency)
        elif ComboBox ==  self.SWCB_4:
            self.diff_3 = (ProfitLoss, currency)
        elif ComboBox ==  self.SWCB_5:
            self.diff_4 = (ProfitLoss, currency)
        else :
            self.diff_5 = (ProfitLoss, currency)
         
        # 輸出損益
        self.output_profit_and_loss()
        
        # 輸出商品結算價差異 EX:60.57 (+0.57)
        diff = ('(+' + f'{diff:.{p_num}f}' + ')') if  diff >= 0 else ('(' + f'{diff:.{p_num}f}' + ')')
        Label.setText(f'{price:.{p_num}f}{diff}')
        
        # 設定顏色格式 
        if SpinBox.value() > 0:                                      
            Label.setStyleSheet("QLabel{border:1px; border-style:solid; color:red}")
        elif SpinBox.value() < 0:
            Label.setStyleSheet("QLabel{border:1px; border-style:solid; color:green}")
        else:
            Label.setStyleSheet("QLabel{border:1px; border-style:solid; color:black}")
            Label.setText(f'{price:.{p_num}f}')                      # 差異為 0             

            
        
#-------------------------------------------------------------------------------------------------------------#       
        # 輸出損益    
    def output_profit_and_loss(self):
        
        # 累積損益初始化
        usd_diff, jpy_diff, eud_diff, hkd_diff= 0, 0, 0, 0
        
        # 累積損益計算 ( 分幣別 )
        for i in (self.diff_1, self.diff_2, self.diff_3, self.diff_4, self.diff_5):
            if i[0] == 0: continue #無損益直接下一圈
            if i[1] == 'USD': usd_diff += i[0]
            elif i[1] == 'EUD' : eud_diff += i[0]
            elif i[1] == 'HKD' : hkd_diff += i[0]
            else : jpy_diff += i[0]      
        print(f'美金損益 : {usd_diff},日圓損益 : {jpy_diff},歐元損益 : {eud_diff},港幣損益 : {hkd_diff}')
        
        datetime = self.SWLB_D1.text()
        broker = self.SWCB_1.currentText()                           
        
        # 輸出調整後權益數 (損益等於 0 直接輸出未調整相同字串)
        if usd_diff != 0:
            usd_for_use = df.Equ_Mar_Ratio_Each('USD', datetime, broker, usd_diff)
            self.SWLB_U4.setText(f'{usd_for_use[0]:,d} / {usd_for_use[1]:,d}  ({usd_for_use[2]})')
            
            if usd_diff > 0: 
                self.SWLB_U5.setStyleSheet("QLabel{color:red}")
                self.SWLB_U5.setText(f' ( +{usd_diff:,d} )')
            else:
                self.SWLB_U5.setStyleSheet("QLabel{color:green}")
                self.SWLB_U5.setText(f' ( {usd_diff:,d} )')
        else:
            self.SWLB_U4.setText(self.SWLB_U2.text())     
            self.SWLB_U5.setText(f' ')
            self.SWLB_U5.setStyleSheet("QLabel{color:black}")
            
        if jpy_diff != 0:
            jpy_for_use = df.Equ_Mar_Ratio_Each('USD', datetime, broker, jpy_diff)
            self.SWLB_J4.setText(f'{jpy_for_use[0]:,d} / {jpy_for_use[1]:,d}  ({jpy_for_use[2]})')
            
            if jpy_diff > 0: 
                self.SWLB_J5.setStyleSheet("QLabel{color:red}")
                self.SWLB_J5.setText(f' ( +{jpy_diff:,d} )')
            else:
                self.SWLB_J5.setStyleSheet("QLabel{color:green}")
                self.SWLB_J5.setText(f' ( {jpy_diff:,d} )')
        else:
            self.SWLB_J4.setText(self.SWLB_J2.text())
            self.SWLB_J5.setText(f' ')
            self.SWLB_J5.setStyleSheet("QLabel{color:black}")

        if eud_diff != 0:
            eud_for_use = df.Equ_Mar_Ratio_Each('USD', datetime, broker, eud_diff)            
            self.SWLB_E4.setText(f'{eud_for_use[0]:,d} / {eud_for_use[1]:,d}  ({eud_for_use[2]})')
            
            if eud_diff > 0: 
                self.SWLB_E5.setStyleSheet("QLabel{color:red}")
                self.SWLB_E5.setText(f' ( +{eud_diff:,d} )')
            else:
                self.SWLB_E5.setStyleSheet("QLabel{color:green}")
                self.SWLB_E5.setText(f' ( {eud_diff:,d} )')
        else:
            self.SWLB_E4.setText(self.SWLB_E2.text())
            self.SWLB_E5.setText(f' ')
            self.SWLB_E5.setStyleSheet("QLabel{color:black}")
                                 
        if hkd_diff != 0:
            hkd_for_use = df.Equ_Mar_Ratio_Each('USD', datetime, broker, hkd_diff)    
            self.SWLB_H4.setText(f'{hkd_for_use[0]:,d} / {hkd_for_use[1]:,d}  ({hkd_for_use[2]})')
            
            if hkd_diff > 0: 
                self.SWLB_H5.setStyleSheet("QLabel{color:red}")
                self.SWLB_H5.setText(f' ( +{hkd_diff:,d} )')
            else:
                self.SWLB_H5.setStyleSheet("QLabel{color:green}")
                self.SWLB_H5.setText(f' ( {hkd_diff:,d} )')
        else:
            self.SWLB_H4.setText(self.SWLB_H2.text())
            self.SWLB_H5.setText(f' ')
            self.SWLB_H5.setStyleSheet("QLabel{color:black}")
        print('輸出完畢')
        
###############################################################################################################  
if __name__ == "__main__":
    app = QApplication(sys.argv)
    MW = MyMain()
    SWP = WidgetP()
    SWR = WidgetR()
    
    MW.MWPB_1.clicked.connect(SWP.show) #視窗1的開啟視窗按鈕
    MW.MWPB_2.clicked.connect(SWR.show) #視窗1的開啟視窗按鈕
    MW.show()
    sys.exit(app.exec_())



SystemExit: 0

c:\users\op1\appdata\local\programs\python\python38-32\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

